In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
DATA_PATH = "/content/processed_flight_records.csv"

df = pd.read_csv(DATA_PATH)

# Identify datetime column
date_cols = [c for c in df.columns if 'date' in c.lower() or 'time' in c.lower()]
date_col = date_cols[0]

df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col)

print("Using datetime column:", date_col)
print("Dataset shape:", df.shape)

In [ ]:
df['date'] = df[date_col].dt.date
df['year'] = df[date_col].dt.year
df['month'] = df[date_col].dt.to_period('M')
df['week'] = df[date_col].dt.to_period('W')
df['day_of_week'] = df[date_col].dt.day_name()

In [ ]:
daily_flights = df.groupby('date').size()
weekly_flights = df.groupby('week').size()
monthly_flights = df.groupby('month').size()

plt.figure()
daily_flights.plot(alpha=0.6)
plt.title("Daily Flight Volume")
plt.xlabel("Date")
plt.ylabel("Flights")
plt.show()

plt.figure()
weekly_flights.plot()
plt.title("Weekly Flight Volume")
plt.xlabel("Week")
plt.ylabel("Flights")
plt.show()

plt.figure()
monthly_flights.plot()
plt.title("Monthly Flight Volume")
plt.xlabel("Month")
plt.ylabel("Flights")
plt.show()

In [ ]:
dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dow_flights = df['day_of_week'].value_counts().reindex(dow_order)

plt.figure()
dow_flights.plot(kind='bar')
plt.title("Flights by Day of Week")
plt.xlabel("Day")
plt.ylabel("Flights")
plt.show()

In [ ]:
month_order = range(1,13)
df['month_num'] = df[date_col].dt.month
month_flights = df.groupby('month_num').size().reindex(month_order)

plt.figure()
month_flights.plot(kind='bar')
plt.title("Flights by Month")
plt.xlabel("Month")
plt.ylabel("Flights")
plt.show()

In [ ]:
# Convert to time series with frequency
monthly_ts = monthly_flights.dropna()
monthly_ts.index = pd.to_datetime(monthly_ts.index, format='%m', errors='ignore')

# Ensure enough data points
if len(monthly_ts) >= 24:
    decomposition = seasonal_decompose(monthly_flights, model='additive', period=12)
    decomposition.plot()
    plt.show()

In [ ]:
heatmap_data = df.pivot_table(index='year', columns='month_num', values='date', aggfunc='count')

plt.figure(figsize=(14,6))
sns.heatmap(heatmap_data, cmap='Blues')
plt.title("Flight Volume Heatmap (Year vs Month)")
plt.xlabel("Month")
plt.ylabel("Year")
plt.show()

In [ ]:
rolling_monthly = monthly_flights.rolling(window=3).mean()

plt.figure()
monthly_flights.plot(label='Actual')
rolling_monthly.plot(label='3-Month Rolling Avg')
plt.legend()
plt.title("Monthly Flights with Rolling Average")
plt.show()

In [ ]:
summary = pd.DataFrame({
    'monthly_flights': monthly_flights,
    'rolling_3mo_avg': rolling_monthly
})

summary.head()

summary.to_csv("/content/time_trends_seasonality_summary.csv")
print("Time trends and seasonality outputs saved.")